## Load numpy

In [ ]:
import numpy as np

### define "euclidean distance" function and "signle link function"

In [50]:
def lp_mat(X,Y,p=2):
    '''
    this is a "Dissimilarity Measure"
    lp_DM_mat(X,Y,1) is equal :
    >>> from sklearn.metrics.pairwise import manhattan_distances
    >>> manhattan_distances(X,Y,sum_over_features=True))
    
    outputs:
    distanceP: n_sample_X by n_sample_Y matrix

    example:
    >>> X = np.ones([1,2])
    >>> Y = 2*np.ones([2,2])
    >>> lp_DM_mat(X,Y)
    [[2,2]]
    '''
    assert p>0, 'p must bigger than 0'
    D = X[:, np.newaxis, :] - Y[np.newaxis, :, :]
    absMat = np.power(np.abs(D),p)
    sumAbsMat = absMat.sum(2)
    distanceP = np.power(sumAbsMat,1.0/p)
    return distanceP

def _single_link(dCiCs,dCjCs,dCiCj, isSim = False, *args, **kwds):
    dCqCs = max(dCiCs,dCjCs) if isSim else min(dCiCs,dCjCs)
    return dCqCs

def _complete_link(dCiCs,dCjCs,dCiCj, isSim = False, *args, **kwds):
    dCqCs = min(dCiCs,dCjCs) if isSim else max(dCiCs,dCjCs)
    return dCqCs

def _wpgma(dCiCs,dCjCs,*args,**kwds):
    return np.mean([dCiCs,dCjCs])

def _upgma(dCiCs,dCjCs,dCiCj,Ci,Cj,yLi,*args,**kwds):
    ni,nj = len(yLi[Ci]),len(yLi[Cj])
    return ni/(ni+nj)*dCiCs+ nj/(ni+nj)*dCjCs

def _upgmc(dCiCs,dCjCs,dCiCj,Ci,Cj,yLi,*args,**kwds):
    ni,nj = len(yLi[Ci]),len(yLi[Cj])
    sumNiNj = ni+nj
    return ni/sumNiNj*dCiCs+nj/sumNiNj*dCjCs-ni*nj/sumNiNj/sumNiNj*dCiCj

def _wpgmc(dCiCs,dCjCs,dCiCj,*args,**kwds):
    return 0.5*dCiCs+0.5*dCjCs-0.25*dCiCj

_funcDi = {
    'single':_single_link,
    'complete':_complete_link,
    'wpgma':_wpgma,
    'upgma':_upgma,
    'upgmc':_upgmc
}

define the MUAS class, which can compatible other algorithm by modify a little code

In [54]:
class MUAS(object):
    '''
    Matrix Updating Algorithmic Scheme
    '''
    def __init__(self,n_component = None,affinity="euclidean",linkage = 'single'):
        self.n_component = n_component       
        self.affinity = affinity
        self.func = _funcDi[linkage]
        
    def fit(self, X, y = None):
        if self.affinity == 'precomputed':
            self.oldP = X
        else:
            self.oldP = lp_mat(X,X,2)
        
        row,col = self.oldP.shape
        self.oldP = np.tri(row,col,-1).T*self.oldP#assign the below tri matrix value 0
        self.yLi = [[i]  for i in  np.arange(row)]# each row(label) has each's point set
        
        breakFlag = True
        while breakFlag:
            
            row,col = self.oldP.shape
            if row == 2: breakFlag = False
             
            #find the mininze value in oldP
            point = np.unique(self.oldP)[1]
            #find the i and j, which will be into q            
            Ci,Cj = np.where(self.oldP == point)
            Ci,Cj = Ci[0],Cj[0]

            Cq,Ctmp = (Ci,Cj) if Ci<Cj else (Cj,Ci)
            
            #extend later points set into before points set
            self.yLi[Cq].extend(self.yLi[Ctmp])
            self.yLi.pop(Ctmp)
            
            self.newP = self.oldP.copy()
            
            #update the sim matrix on the origin matrix structure
            for i in range(1,row):
                if i==Ci or i==Cj: continue
                
                self.newP[Cq,i] = self.func(self.oldP[Ci,i],self.oldP[Cj,i],self.oldP[Ci,Cj],Ci,Cj,self.yLi)
            #delete the Ctmp row and col
            self.newP = np.delete(self.newP,Ctmp,0)
            self.newP = np.delete(self.newP,Ctmp,1)    
            self.oldP = self.newP
            
            if len(self.yLi) == self.n_component: break 
        return self.yLi           

In [55]:
X = np.array([[0, 1, 2, 26, 37],
             [1, 0, 3, 25, 36],
             [2, 3, 0, 16, 25],
             [26,25,16,0,  1.5],
             [37,36,25,1.5,0]])

muas = MUAS(2,affinity = 'precomputed')
y = muas.fit(X)

1.0
1.5
3.0


In [53]:
y

[[0, 1, 2], [3, 4]]